## PyCity Schools Analysis
###### This analysis displays metrics taken from 15 different high schools and cross referencing two key factors: budget and test scores across schools and students. Two separate data frames were given: a data frame for school information and a data frame for student information. These two date frames were inner merged into one data frame, then subsequently analyzed. The data was first analyzed at a high level into a district summary, then, the data was grouped by schools and school specific data was analyzed further by various characteristics. The following conclusions were drawn from this data analysis:
###### 1. Larger high schools consisting of between 2000-5000 students appear to perform worse in math and reading overall than high schools with less than 2000 students. In reference to the data frame organizing data by school size, the average math scores, reading scores, and average passing rates across students are notably lower in large high schools (2000-5000 students) than medium (1000-2000 students) and small (<1000 students) high schools. (See Cell 41) 
###### 2. Students from charter high schools appear to perform better in math and reading overall than students from district high schools. In reference to the data frame organizing data by type of school, the average math scores, reading scores, and average passing rates across students are notably higher in charter schools than district schools. (See Cell 44) This may suggest that 

In [23]:
import pandas as pd
from pathlib import Path

# file path
schools_complete_csv = Path('Resources/schools_complete.csv')
students_complete_csv = Path('Resources/students_complete.csv')

# read file
students_df = pd.read_csv(students_complete_csv)
schools_df = pd.read_csv(schools_complete_csv)

# merge data frames
schools_students_df = pd.merge(students_df, schools_df)

# preview merged data frame
schools_students_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [24]:
# total unique schools
total_schools = schools_df['School ID'].nunique()
total_schools

15

In [25]:
# total number of students
total_students = students_df['Student ID'].nunique()
total_students = int(total_students)
total_students

39170

In [26]:
# total budget
total_budget = schools_df['budget'].sum()
total_budget

24649428

In [27]:
# average math score
avg_math_score = students_df['math_score'].mean()
avg_math_score

78.98537145774827

In [28]:
# average reading score
avg_read_score = students_df['reading_score'].mean()
avg_read_score

81.87784018381414

In [29]:
# count of students passing math
count_pass_math = students_df[(students_df['math_score'] >= 70)]['student_name'].count()

# percentage of students passing math
percent_pass_math = count_pass_math / int(total_students) * 100
percent_pass_math

74.9808526933878

In [30]:
# count of students passing reading
count_pass_read = students_df[(students_df['reading_score'] >= 70)]['student_name'].count()

# percentage of students passing reading
percent_pass_read = count_pass_read / int(total_students) * 100
percent_pass_read

85.80546336482001

In [31]:
# count of students passing math and reading
count_pass_overall = students_df[
        (students_df['reading_score'] >= 70) &
        (students_df['math_score'] >= 70)]['student_name'].count()

# percentage of students passing math and reading
percent_pass_overall = count_pass_overall / int(total_students) * 100
percent_pass_overall

65.17232575950983

In [32]:
# district's key metrics summary

# create data frame with dictionary
district_summary_df = pd.DataFrame({
    'total_schools': [total_schools], 
    'total_students': [total_students],
    'total_budget': [total_budget],
    'average_math_score': [avg_math_score],
    'avg_reading_score': [avg_read_score],
    'percent_passing_math': [percent_pass_math],
    'percent_passing_reading': [percent_pass_read],
    'percent_overall_passing': [percent_pass_overall]})

# format values
district_summary_df['total_students'] = district_summary_df['total_students'].map('{:,}'.format)
district_summary_df['total_budget'] = district_summary_df['total_budget'].map('${:,.2f}'.format)

# display data frame
district_summary_df

,total_schools,total_students,total_budget,average_math_score,avg_reading_score,percent_passing_math,percent_passing_reading,percent_overall_passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [33]:
# group by schools
sch_grouping = schools_students_df.set_index('school_name').groupby(['school_name'])

In [34]:
# school types
sch_type = schools_df.set_index(['school_name'])['type']

In [35]:
# students per school
students_per_sch = sch_grouping['Student ID'].count()

In [36]:
# budget per school
budget_per_sch = schools_df.set_index(['school_name'])['budget']

# school budget per capita
sch_budget_per_capita = schools_df.set_index(['school_name'])['budget'] / schools_df.set_index(['school_name'])['size']

In [37]:
# average math and reading scores per school
avg_math_per_sch = sch_grouping['math_score'].mean()
avg_read_per_sch = sch_grouping['reading_score'].mean()

In [38]:
# students passing math per school
pass_math_per_sch = schools_students_df[schools_students_df['math_score'] >= 70] \
    .groupby('school_name')['Student ID'].count() 

# students passing reading per school
pass_read_per_sch = schools_students_df[schools_students_df['reading_score'] >=70] \
    .groupby('school_name')['Student ID'].count() 

# students passing both per school
pass_overall_per_sch = schools_students_df[
    (schools_students_df['math_score'] >= 70) & 
    (schools_students_df['reading_score'] >= 70)
].count()

In [39]:
# math passing rates per school
pass_math_rate = schools_students_df[schools_students_df['math_score'] >= 70] \
    .groupby('school_name')['Student ID'].count() / students_per_sch * 100

# reading passing rates per school
pass_read_rate = schools_students_df[schools_students_df['reading_score'] >= 70] \
    .groupby('school_name')['Student ID'].count() / students_per_sch * 100

# overall passing rates per school
pass_both_rate = schools_students_df[
    (schools_students_df['math_score'] >= 70) & 
    (schools_students_df['reading_score'] >= 70)
].groupby('school_name')['Student ID'].count() / students_per_sch * 100

In [40]:
# per school summary data frame

# create dictionary
school_summary_df = pd.DataFrame({
    'school_type': sch_type, 
    'total_students': students_per_sch,
    'total_school_budget': budget_per_sch,
    'per_student_budget': sch_budget_per_capita,
    'average_math_score': avg_math_per_sch,
    'average_reading_score': avg_read_per_sch,
    'percent_passing_math': pass_math_rate,
    'percent_passing_reading': pass_read_rate,
    'percent_passing_overall': pass_both_rate})

# format values
school_summary_df['total_school_budget'] = school_summary_df['total_school_budget'].map('${:,.2f}'.format)

# display data frame
school_summary_df.index.name = None
school_summary_df

,school_type,total_students,total_school_budget,per_student_budget,average_math_score,average_reading_score,percent_passing_math,percent_passing_reading,percent_passing_overall
Bailey High School,District,4976,"$3,124,928.00",628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",650.0,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## Highest Performing Schools (By % Overall Passing)

In [57]:
# highest performing schools, in descending order
highest_performing = school_summary_df.sort_values('percent_passing_overall', ascending=False)

# format values
highest_performing['per_student_budget'] = highest_performing['per_student_budget'].map('${:,.2f}'.format)

# display data frame
highest_performing.index.name = None
highest_performing.head()

,school_type,total_students,total_school_budget,per_student_budget,average_math_score,average_reading_score,percent_passing_math,percent_passing_reading,percent_passing_overall
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Lowest Performing Schools (By % Overall Passing)

In [58]:
# lowest performing schools, in ascending order
lowest_performing = school_summary_df.sort_values('percent_passing_overall', ascending=True)

# format values
lowest_performing['per_student_budget'] = lowest_performing['per_student_budget'].map('${:,.2f}'.format)

# display data frame
lowest_performing.index.name = None
lowest_performing.head()

,school_type,total_students,total_school_budget,per_student_budget,average_math_score,average_reading_score,percent_passing_math,percent_passing_reading,percent_passing_overall
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [43]:
# separate scores by grade
ninth_grade = schools_students_df[(schools_students_df['grade'] == '9th')]
tenth_grade = schools_students_df[(schools_students_df['grade'] == '10th')]
eleventh_grade = schools_students_df[(schools_students_df['grade'] == '11th')]
twelfth_grade = schools_students_df[(schools_students_df['grade'] == '12th')]

# average math scores per school
ninth_math_scores = students_df.loc[students_df['grade'] == '9th'] \
    .groupby('school_name')['math_score'].mean()
tenth_math_scores = students_df.loc[students_df['grade'] == '10th'] \
    .groupby('school_name')['math_score'].mean()
eleventh_math_scores = students_df.loc[students_df['grade'] == '11th'] \
    .groupby('school_name')['math_score'].mean()
twelfth_math_scores = students_df.loc[students_df['grade'] == '9th'] \
    .groupby('school_name')['math_score'].mean()

# create data frame
math_scores_by_grade = pd.DataFrame({
    '9th': ninth_math_scores,
    '10th': tenth_math_scores,
    '11th': eleventh_math_scores,
    '12th': twelfth_math_scores})

# display data frame
math_scores_by_grade.index.name = None
math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,77.083676
Cabrera High School,83.094697,83.154506,82.765560,83.094697
Figueroa High School,76.403037,76.539974,76.884344,76.403037
Ford High School,77.361345,77.672316,76.918058,77.361345
Griffin High School,82.044010,84.229064,83.842105,82.044010
Hernandez High School,77.438495,77.337408,77.136029,77.438495
Holden High School,83.787402,83.429825,85.000000,83.787402
Huang High School,77.027251,75.908735,76.446602,77.027251
Johnson High School,77.187857,76.691117,77.491653,77.187857
Pena High School,83.625455,83.372000,84.328125,83.625455


## Reading Scores by Grade

In [44]:
# average reading scores per school by grade
ninth_read_scores = students_df.loc[students_df['grade'] == '9th'] \
    .groupby('school_name')['reading_score'].mean()
tenth_read_scores = students_df.loc[students_df['grade'] == '10th'] \
    .groupby('school_name')['reading_score'].mean()
eleventh_read_scores = students_df.loc[students_df['grade'] == '11th'] \
    .groupby('school_name')['reading_score'].mean()
twelfth_read_scores = students_df.loc[students_df['grade'] == '9th'] \
    .groupby('school_name')['reading_score'].mean()

# create data frame
reading_scores_by_grade = pd.DataFrame({
    '9th': ninth_read_scores,
    '10th': tenth_read_scores,
    '11th': eleventh_read_scores,
    '12th': twelfth_read_scores})

# display data frame
reading_scores_by_grade.index.name = None
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,81.303155
Cabrera High School,83.676136,84.253219,83.788382,83.676136
Figueroa High School,81.198598,81.408912,80.640339,81.198598
Ford High School,80.632653,81.262712,80.403642,80.632653
Griffin High School,83.369193,83.706897,84.288089,83.369193
Hernandez High School,80.866860,80.660147,81.396140,80.866860
Holden High School,83.677165,83.324561,83.815534,83.677165
Huang High School,81.290284,81.512386,81.417476,81.290284
Johnson High School,81.260714,80.773431,80.616027,81.260714
Pena High School,83.807273,83.612000,84.335938,83.807273


## Scores by School Spending

In [60]:
# establish bins
spending_bins = [0, 585, 630, 645, 680]
labels = ['<$585', '$585-630', '$630-645', '$645-680']

In [61]:
# create copy of school summary
school_spending_df = school_summary_df.copy()

In [62]:
# categorize spending based on bins
school_spending_df['spending_ranges_per_student'] = pd.cut(school_spending_df['per_student_budget'], spending_bins, labels=labels)

# format data
school_spending_df['per_student_budget'] = school_spending_df['per_student_budget'].map('${:,.2f}'.format)

# display data frame
school_spending_df

,school_type,total_students,total_school_budget,per_student_budget,average_math_score,average_reading_score,percent_passing_math,percent_passing_reading,percent_passing_overall,spending_ranges_per_student
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,$585-630
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,<$585
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,$630-645
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,$630-645
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,$585-630
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,$645-680
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166,<$585
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,$645-680
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172,$645-680
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541,$585-630


In [48]:
# spending averages
avg_spend_math_scores = school_spending_df \
    .groupby(['spending_ranges_per_student'], observed=False)["average_math_score"].mean() 
avg_spend_read_scores = school_spending_df \
    .groupby(['spending_ranges_per_student'], observed=False)["average_reading_score"].mean()
avg_spend_pass_math = school_spending_df \
    .groupby(['spending_ranges_per_student'], observed=False)["percent_passing_math"].mean()
avg_spend_pass_read = school_spending_df \
    .groupby(['spending_ranges_per_student'], observed=False)["percent_passing_reading"].mean()
avg_overall_pass_spend = school_spending_df \
    .groupby(['spending_ranges_per_student'], observed=False)["percent_passing_overall"].mean()

In [49]:
# scores by school spending data frame
spending_summary_df = pd.DataFrame({
    "average_math_score": avg_spend_math_scores,
    "average_reading_score": avg_spend_read_scores,
    "percent_passing_math": avg_spend_pass_math,
    "percent_passing_reading": avg_spend_pass_read,
    "percent_overall_passing": avg_overall_pass_spend})

# display data frame
spending_summary_df

,average_math_score,average_reading_score,percent_passing_math,percent_passing_reading,percent_overall_passing
spending_ranges_per_student,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

In [64]:
# establish bins
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [65]:
# create copy of school summary
school_size_df = school_summary_df.copy()

In [67]:
# categorize school size based on bins
school_size_df['school_size'] = pd.cut(school_size_df['total_students'], size_bins, labels=labels)

# format data
school_size_df['per_student_budget'] = school_size_df['per_student_budget'].map('${:,.2f}'.format)

# display data frame
school_size_df

,school_type,total_students,total_school_budget,per_student_budget,average_math_score,average_reading_score,percent_passing_math,percent_passing_reading,percent_passing_overall,school_size
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,Medium (1000-2000)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,Large (2000-5000)
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,Medium (1000-2000)
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,Large (2000-5000)
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166,Small (<1000)
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,Large (2000-5000)
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172,Large (2000-5000)
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541,Small (<1000)


In [53]:
# size averages
avg_size_math_scores = school_size_df \
    .groupby(['school_size'], observed=False)['average_math_score'].mean()
avg_size_read_scores = school_size_df \
    .groupby(['school_size'], observed=False)['average_reading_score'].mean()
avg_size_pass_math = school_size_df \
    .groupby(['school_size'], observed=False)['percent_passing_math'].mean()
avg_size_pass_read = school_size_df \
    .groupby(['school_size'], observed=False)['percent_passing_reading'].mean()
avg_size_pass_both = school_size_df \
    .groupby(['school_size'], observed=False)['percent_passing_overall'].mean()

In [54]:
# summary based on school size
size_summary_df = pd.DataFrame({
    'average_math_score': avg_size_math_scores,
    'average_reading_score': avg_size_read_scores,
    'percent_passing_math': avg_size_pass_math,
    'percent_passing_reading': avg_size_pass_read,
    'percent_overall_passing': avg_size_pass_both})

# display data frame
size_summary_df

,average_math_score,average_reading_score,percent_passing_math,percent_passing_reading,percent_overall_passing
school_size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

In [55]:
# summary based on school type
avg_math_score_type = school_summary_df \
    .groupby(['school_type'])['average_math_score'].mean()
avg_read_score_type = school_summary_df \
    .groupby(['school_type'])['average_reading_score'].mean()
avg_pass_math_type = school_summary_df \
    .groupby(['school_type'])['percent_passing_math'].mean()
avg_pass_read_type = school_summary_df \
    .groupby(['school_type'])['percent_passing_reading'].mean()
avg_overall_pass_type = school_summary_df \
    .groupby(['school_type'])['percent_passing_overall'].mean()

In [56]:
# summary based on type data
type_summary_df = pd.DataFrame({
    "average_math_score": avg_math_score_type,
    "average_reading_score": avg_read_score_type,
    "percent_passing_math": avg_pass_math_type,
    "percent_passing_reading": avg_pass_read_type,
    "percent_overall_passing": avg_overall_pass_type})

# display data frame
type_summary_df

,average_math_score,average_reading_score,percent_passing_math,percent_passing_reading,percent_overall_passing
school_type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
